In [2]:
import pandas as pd
import numpy as np
import geopy.distance
from itertools import product
from tqdm import tqdm

In [3]:

from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [4]:
colex = pd.read_csv("../data/colexifications/colex_all_dedup.csv")

In [7]:
colex_langs = set(list(colex["Glottocode"]))

In [9]:
len(colex_langs)

2933

In [5]:
phon = pd.read_csv("../data/stage1/pmiLanguageDistances.csv",sep="\t")

In [12]:
phon_langs = set(phon.Glottocode1.tolist()+phon.Glottocode2.tolist())

In [15]:
len(phon_langs)

1563

In [16]:
all_langs = phon_langs.union(colex_langs)

In [17]:
len(all_langs)

2933

In [18]:
len(phon_langs.intersection(colex_langs))

1563

# Step1: Write lang2lang coords to csv

In [20]:
dflang = pd.read_csv("../data/languages/languages_all.csv", index_col=0)

In [30]:
dflang.head()

,parent,name,level,hid,iso639_3,latitude,longitude,macroareas,countries,timespan,branch,parent_name,id.1,family,family_id,macroarea,nr_macroarea
id,,,,,,,,,,,,,,,,,
abin1243,NaN,Abinomn,language,bsa,bsa,-2.922810,138.891000,['Papunesia'],['Indonesia'],NaN,NaN,NaN,abin1243,NaN,NaN,Papunesia,1
abis1238,NaN,Aewa,language,ash,ash,-1.284096,-75.084405,['South America'],['Peru'],NaN,NaN,NaN,abis1238,NaN,NaN,South America,1
abaz1241,abkh1243,Abaza,language,abq,abq,44.250000,42.000000,['Eurasia'],"['Russian Federation', 'Turkey']",NaN,Abkhaz-Abaza,Abkhaz-Abaza,abaz1241,Abkhaz-Adyge,abkh1242,Eurasia,1
abkh1244,abkh1243,Abkhaz,language,abk,abk,43.056218,41.159115,['Eurasia'],"['Georgia', 'Russian Federation', 'Turkey']",NaN,Abkhaz-Abaza,Abkhaz-Abaza,abkh1244,Abkhaz-Adyge,abkh1242,Eurasia,1
adyg1241,circ1239,Adyghe,language,ady,ady,44.000000,39.330000,['Eurasia'],"['Israel', 'Jordan', 'Russian Federation', 'Sy...",NaN,Circassian,Circassian,adyg1241,Abkhaz-Adyge,abkh1242,Eurasia,1


In [27]:
dflang.dropna(subset=["latitude", "longitude"], inplace=True)

In [28]:
coords = {lang:(lat, long) for (lang, lat,long) in zip(dflang.index,dflang["latitude"], dflang["longitude"])}

In [29]:
len(coords)

8763

In [36]:
langs = list(coords.keys())
len(langs)

8763

In [42]:
interlangs = set(langs).intersection(all_langs)

In [43]:
len(interlangs)

2554

In [44]:
combs = product(interlangs, interlangs, repeat=1)

In [38]:
writer = open("../data/stage1/lang2lang_coords.csv","w+")
writer.write("Glottocode1\tCoord1\tGlottocode2\tCoord2\n")

38

In [45]:

for lang2lang in tqdm(list(combs)):
    lang1, lang2= lang2lang
    # print(lang1,lang2)
    coord1 = coords[lang1]
    coord2= coords[lang2]
    writer.write((f"{lang1}\t{coord1}\t{lang2}\t{coord2}\n"))

100%|█████████████████████████████| 6522916/6522916 [00:14<00:00, 454162.14it/s]


# Step2: calculate lang2lang distance parallelly

In [47]:
from ast import literal_eval

In [48]:
def geodesic_dist(coord1, coord2):
    return geopy.distance.geodesic(coord1,coord2).km

In [49]:
df = pd.read_csv("../data/stage1/lang2lang_coords.csv", sep="\t")

In [51]:
len(df)

6522896

In [50]:
df

,Glottocode1,Coord1,Glottocode2,Coord2
0,mawa1267,"(-5.2219, 145.589)",mawa1267,"(-5.2219, 145.589)"
1,mawa1267,"(-5.2219, 145.589)",paic1239,"(-21.096, 165.15)"
2,mawa1267,"(-5.2219, 145.589)",west2650,"(12.4771, 104.026)"
3,mawa1267,"(-5.2219, 145.589)",anja1238,"(-5.45899, 145.717)"
4,mawa1267,"(-5.2219, 145.589)",hanu1241,"(12.4159, 121.221)"
...,...,...,...,...
6522891,lauu1247,"(-8.51018, 160.872)",cune1239,"(38.0, 36.0)"
6522892,lauu1247,"(-8.51018, 160.872)",pany1241,"(-22.7816, 118.723)"
6522893,lauu1247,"(-8.51018, 160.872)",bara1380,"(0.02193, -70.808)"
6522894,lauu1247,"(-8.51018, 160.872)",cayu1262,"(-13.2482, -65.4822)"


In [52]:
df["Coord1"]=df["Coord1"].parallel_apply(literal_eval)
df["Coord2"]=df["Coord2"].parallel_apply(literal_eval)

In [53]:
df["Coord1"].tolist()[0]

(-5.2219, 145.589)

In [54]:
df["GeodesicDist"]=df[["Coord1", "Coord2"]].parallel_apply(lambda x: geodesic_dist(*x), axis=1)

In [55]:
df.to_csv("../data/stage1/lang2lang_geodesic.csv", index=False)

# Step3: get language contact distance

In [56]:
import json
import os
from collections import defaultdict

In [57]:
lang2lang_dict = dict(zip(zip(df["Glottocode1"],df["Glottocode2"]), df["GeodesicDist"]))

In [58]:
lang2lang_dict_sorted={tuple(sorted(x)):y for x,y in lang2lang_dict.items()}

In [62]:
len(lang2lang_dict_sorted)

3262734

In [68]:
output_folder="../data/stage1/language_contacts_nr1"
os.makedirs(output_folder)

In [100]:
def get_lang2lang_contacts(tgt_lang, worklangs=list(interlangs), lang2lang_dict=lang2lang_dict_sorted):
    outputfile = f"{output_folder}/{tgt_lang}.json"
    if not os.path.exists(outputfile):
        in_dryer2018=defaultdict(dict)
        in_dryer2018[tgt_lang]=dict()
        possible_langs= [x for x in worklangs if x!=tgt_lang]
        for z in possible_langs:
            between_count=0
            y = tgt_lang
            for x in worklangs:
                try:
                    x_y = tuple(sorted([x,y]))
                    x_z = tuple(sorted([x,z]))
                    y_z = tuple(sorted([y,z]))
                    dist_x_y = lang2lang_dict[x_y]
                    dist_x_z = lang2lang_dict[x_z]
                    dist_y_z = lang2lang_dict[y_z]
            
                    if dist_x_y < dist_y_z and dist_x_z <dist_y_z:
                        between_count+=1
                except Exception: 
                    print(x,y,z)
                    continue
            in_dryer2018[tgt_lang][z]= between_count
        with open(outputfile, "w+") as f:
            json.dump(in_dryer2018, f)
    # return in_dryer2018
    

In [ ]:
for lang in tqdm(interlangs):
    get_lang2lang_contacts(lang, interlangs, lang2lang_dict_sorted)

In [77]:
from joblib import Parallel, delayed


In [78]:
lang2lang_dict_sorted[("abau1245", "pyuu1245")]

62.81559433760269

In [83]:
lang2lang_dict_sorted[("lauu1247", "pyuu1245")]

2240.936976501583

In [91]:
len(interlangs)

2554

In [93]:
"lauu1247"=="lauu1247" 

True

In [103]:
Parallel(n_jobs=-1)(delayed(get_lang2lang_contacts)(i) for i in tqdm(interlangs))









  0%|                                                  | 0/2554 [00:00<?, ?it/s]







  0%|▏                                        | 10/2554 [00:01<05:29,  7.71it/s]







  1%|▎                                        | 20/2554 [00:02<04:46,  8.85it/s]







  1%|▍                                        | 30/2554 [00:12<22:35,  1.86it/s]







  2%|▋                                        | 40/2554 [00:21<28:02,  1.49it/s]







  2%|▊                                        | 50/2554 [00:31<33:31,  1.25it/s]







  2%|▉                                        | 60/2554 [00:42<36:23,  1.14it/s]







  3%|█                                        | 70/2554 [00:51<37:31,  1.10it/s]







  3%|█▎                                       | 80/2554 [01:01<38:24,  1.07it/s]







  4%|█▍                                       | 90/2554 [01:11<39:19,  1.04it/s]







  4%|█▌                                      | 100/2554 [01:22<40:19,  1.01it/s]







  4%|█▋                 

lauu1247 gorr1238 lauu1247
lauu1247 ngar1296 lauu1247
lauu1247 oroh1237 lauu1247










  9%|███▌                                    | 230/2554 [03:45<52:23,  1.35s/it]

lauu1247 gudu1252 lauu1247
lauu1247 qaua1234 lauu1247
lauu1247 sene1263 lauu1247
lauu1247 cent2142 lauu1247
lauu1247 moss1236 lauu1247
lauu1247 waya1269 lauu1247










  9%|███▊                                    | 240/2554 [03:59<53:20,  1.38s/it]

...









 99%|████████████████████████████████████▊| 2540/2554 [1:11:40<00:26,  1.86s/it]

lauu1247 ngar1235 lauu1247
lauu1247 mait1250 lauu1247
lauu1247 ngit1239 lauu1247
lauu1247 sout2728 lauu1247










100%|█████████████████████████████████████| 2554/2554 [1:11:57<00:00,  1.69s/it]


lauu1247 duvl1242 lauu1247
lauu1247 helo1243 lauu1247
lauu1247 oira1263 lauu1247
lauu1247 uppe1430 lauu1247
lauu1247 sout2754 lauu1247
lauu1247 bara1380 lauu1247
lauu1247 clas1252 lauu1247
lauu1247 rung1255 lauu1247
lauu1247 nucl1379 lauu1247
lauu1247 cune1239 lauu1247
lauu1247 renn1242 lauu1247
lauu1247 swed1254 lauu1247
lauu1247 ship1254 lauu1247
lauu1247 pany1241 lauu1247
lauu1247 amba1265 lauu1247
lauu1247 waff1241 lauu1247


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

# Step4: Merge language distance dictionaries.

In [105]:
import networkx as nx

In [106]:
input_folder="../data/stage1/language_contacts_nr1"

In [107]:
langs = [x.replace(".json","") for x in os.listdir(input_folder) if x.endswith(".json")]

In [108]:
len(langs)

2554

In [109]:
langs[0]

'guin1254'

## populate a graph

In [113]:
def langauge_info():
    """
    Load language information
    :return: a dictionary of language information with glottocode as key.
    """
    langdict = pd.read_csv("../data/languages/languages_all.csv", index_col=0).to_dict(orient="index")
    return langdict

In [114]:
langdict=langauge_info()

In [151]:
g = nx.Graph()
g.graph["dataset"]="geo"
g.graph["langs"]=len(langs)

In [152]:
g.graph

{'dataset': 'geo', 'langs': 2554}

In [153]:
len(langs)

2554

In [155]:
langdict["guin1254"]

{'parent': 'kpel1252',
 'name': 'Guinea Kpelle',
 'level': 'language',
 'hid': 'gkp',
 'iso639_3': 'gkp',
 'latitude': 7.93196,
 'longitude': -8.98843,
 'macroareas': "['Africa']",
 'countries': "['Guinea', 'Liberia']",
 'timespan': nan,
 'branch': 'Kpelle',
 'parent_name': 'Kpelle',
 'id.1': 'guin1254',
 'family': 'Mande',
 'family_id': 'mand1469',
 'macroarea': 'Africa',
 'nr_macroarea': 1}

In [156]:

for lang in tqdm(langs):
    node_dict = langdict[lang] # geo information
    name = node_dict["name"]
    fam = node_dict["family"]
    parent = node_dict["parent_name"]
    branch = node_dict["branch"]
    iso_code = node_dict["iso639_3"]
    lat = node_dict["latitude"]
    long=node_dict["longitude"]
    area = node_dict["macroarea"]
    times=[]
    if str(node_dict["timespan"])!="nan":
        timespan = literal_eval(node_dict["timespan"])
        start_year= timespan["start_year"]
        times.append(start_year)
        end_year=timespan["end_year"]
        times.append(end_year)
    
    g.add_node(lang, 
                   name=name,
                   family= fam,
                   parent= parent,
                   branch= branch,
                   iso3 = iso_code,
                   area = area,
                   timespan = tuple(times),
                   coord= (lat, long))









100%|███████████████████████████████████| 2554/2554 [00:00<00:00, 184512.68it/s]


In [173]:

    
for lang in tqdm(langs):
    with open(f"{input_folder}/{lang}.json") as f:
        langd = json.load(f)

    for lang2, con in langd[lang].items():
        if con is not None:
            g.add_edge(lang, lang2)
            branch1 = g.nodes[lang]["branch"]
            branch2= g.nodes[lang2]["branch"]
            
            if str(branch1)!="nan" and str(branch2)!="nan":
                if branch1 != branch2:
                    branch=0
                elif branch1 == branch2:
                    branch=1
                else:
                    branch=-1

            else:
                branch=-1
                    
            

            l1,l2= sorted([lang,lang2])
            distkm =lang2lang_dict_sorted[(l1,l2)]
            if con<10:
                g.add_edge(lang,lang2, contact=con, geodist=distkm, neighbour=1, branch=branch)
            else:
                g.add_edge(lang,lang2, contact=con, geodist=distkm, neighbour=0, branch=branch)

            










  0%|                                                  | 0/2554 [00:00<?, ?it/s]







  0%|▏                                         | 9/2554 [00:00<00:30, 83.84it/s]







  1%|▎                                        | 19/2554 [00:00<00:27, 91.50it/s]







  1%|▍                                        | 29/2554 [00:00<00:27, 90.96it/s]







  2%|▋                                        | 40/2554 [00:00<00:26, 95.51it/s]







  2%|▊                                        | 50/2554 [00:00<00:25, 96.41it/s]







  2%|▉                                        | 60/2554 [00:00<00:26, 94.55it/s]







  3%|█                                        | 70/2554 [00:00<00:26, 93.85it/s]







  3%|█▎                                       | 80/2554 [00:00<00:25, 95.16it/s]







  4%|█▍                                       | 91/2554 [00:00<00:25, 98.05it/s]







  4%|█▌                                      | 101/2554 [00:01<00:24, 98.52it/s]







  4%|█▋                 

In [166]:
g.edges['guin1254', 'pawa1255']

{'contact': 2119, 'geodist': 17172.307067554568, 'neighbour': 0, 'branch': -1}

In [159]:
g.edges[ 'pawa1255','guin1254']

{'contact': 2119, 'geodist': 17172.307067554568, 'neighbour': 0, 'branch': -1}

In [160]:
len(g.nodes), len(g.edges)

(2554, 3260181)

In [ ]:

for edge in g.edges:
    t1, t2= edge
    branch = g.edges[t1,t2]["branch"]
    neighbour = g.edges[t1,t2]["neighbour"]
    if branch==1:
        if neighbour!=1:
            print(t1,t2, g.edges[t1,t2])
            

In [199]:
print(g.nodes["iumi1238"])

{'name': 'Iu Mien', 'family': 'Hmong-Mien', 'parent': 'Mien-Mun', 'branch': 'Mien-Mun', 'iso3': 'ium', 'area': 'Eurasia', 'timespan': (), 'coord': (20.83, 101.17)}


In [200]:
print(g.nodes["kimm1245"])

{'name': 'Kim Mun', 'family': 'Hmong-Mien', 'parent': 'Mien-Mun', 'branch': 'Mien-Mun', 'iso3': 'mji', 'area': 'Eurasia', 'timespan': (), 'coord': (22.9856, 105.024)}


In [188]:
g.edges["stan1295", "dutc1256"]

{'contact': 13, 'geodist': 648.8953043040958, 'neighbour': 0, 'branch': 0}

In [190]:
g.edges["loth1237", "sang1321"]

{'contact': 4, 'geodist': 61.02906153629475, 'neighbour': 1, 'branch': 1}

In [191]:
g.nodes["loth1237"], g.nodes["sang1321"]

({'name': 'Lotha Naga',
  'family': 'Sino-Tibetan',
  'parent': 'Central Naga',
  'branch': 'Central Naga',
  'iso3': 'njh',
  'area': 'Eurasia',
  'timespan': (),
  'coord': (26.01396, 94.04355)},
 {'name': 'Sangtam Naga',
  'family': 'Sino-Tibetan',
  'parent': 'Central Naga',
  'branch': 'Central Naga',
  'iso3': 'nsa',
  'area': 'Eurasia',
  'timespan': (),
  'coord': (25.8581, 94.6279)})

In [178]:
list(g.subgraph(["stan1295"]).edges)

[]

In [182]:
g.subgraph(["stan1295", "nort2627", "xhos1239"]).edges

EdgeView([('nort2627', 'xhos1239'), ('nort2627', 'stan1295'), ('xhos1239', 'stan1295')])

In [161]:
for node in g.nodes:
    if "German" in g.nodes[node]["name"]:
        print(node, g.nodes[node])

nort2627 {'name': 'Eastern Low German', 'family': 'Indo-European', 'parent': 'Greater East Low German', 'branch': 'Greater East Low German', 'iso3': 'nds', 'area': 'Eurasia', 'timespan': (), 'coord': (53.115373, 12.391439)}
stan1295 {'name': 'German', 'family': 'Indo-European', 'parent': 'Global German', 'branch': 'Global German', 'iso3': 'deu', 'area': 'Eurasia', 'timespan': (), 'coord': (48.649, 12.4676)}
oldh1241 {'name': 'Old High German (ca. 750-1050)', 'family': 'Indo-European', 'parent': 'Upper German', 'branch': 'Upper German', 'iso3': 'goh', 'area': 'Eurasia', 'timespan': (800, 1100), 'coord': (52.0, 10.0)}
midd1343 {'name': 'Middle High German', 'family': 'Indo-European', 'parent': 'Middle-Modern High German', 'branch': 'Middle-Modern High German', 'iso3': 'gmh', 'area': 'Eurasia', 'timespan': (1050, 1500), 'coord': (48.254, 10.599)}
midd1318 {'name': 'Middle Low German', 'family': 'Indo-European', 'parent': 'Middle-Modern Low German', 'branch': 'Middle-Modern Low German', 'i

In [ ]:
for node in g.nodes:
    

In [144]:
for node in g.nodes:
    if "Dutch" in g.nodes[node]["name"]:
        print(node, g.nodes[node])

dutc1256 {'name': 'Dutch', 'family': 'Indo-European', 'parent': 'Global Dutch', 'branch': 'Global Dutch', 'iso3': 'nld', 'area': 'Eurasia', 'timespan': (), 'coord': (52.0, 5.0)}
midd1321 {'name': 'Middle Dutch', 'family': 'Indo-European', 'parent': 'Middle-Modern Dutch', 'branch': 'Middle-Modern Dutch', 'iso3': 'dum', 'area': 'Eurasia', 'timespan': (1100, 1500), 'coord': (51.656, 5.34)}


In [148]:
nx.subgraph(g, ["stan1295", "dutc1256"])

In [145]:
g.edges["stan1295", "dutc1256"]

{'contact': 13, 'geodist': 648.8953043040958, 'neighbour': 0}

In [139]:
g.nodes["pawa1255"]

{'name': 'Pawaia',
 'family': nan,
 'parent': nan,
 'branch': nan,
 'iso3': 'pwa',
 'area': 'Papunesia',
 'timespan': (),
 'coord': (-6.88021, 145.081)}

In [185]:
g= g.to_undirected(as_view=True)

In [186]:
len(g.nodes), len(g.edges)

(2554, 3260181)

In [187]:
import pickle
with open("../data/stage1/language_geo_graph.pickle", "wb") as f:
    pickle.dump(g,f)

# Others

In [15]:
langs= pd.read_csv("../data/stage1/pmiLanguageDistances.csv", sep="\t")

In [18]:
langs_=langs.dropna(subset=["PMI"])

In [22]:
glottocodes= list(set(langs_["Glottocode1"].tolist()+langs_["Glottocode2"].tolist()))

In [24]:
len(glottocodes)

1563

In [63]:
glottocodes.remove("hang1263")

In [5]:
df_langs = pd.read_csv("../data/languages/languages_family_area.csv")

In [7]:
df_langs=df_langs.dropna(subset=["latitude","longitude"])


In [25]:
df_langs = df_langs[df_langs.id.isin(glottocodes)]

In [45]:
df_langs["macroarea"].value_counts()

macroarea
Papunesia        605
Africa           381
Eurasia          258
South America    173
North America    140
Australia          4
Name: count, dtype: int64

In [46]:
df_langs[df_langs["macroarea"]=="Australia"]

,id,name,family_id,top-level family,macroarea,latitude,longitude,iso639-3
9343,yuga1244,Yugambal,pama1250,Pama-Nyungan,Australia,-27.8030,153.082,yub
10178,maun1240,Mawng,iwai1246,Iwaidjan Proper,Australia,-11.6600,133.376,mph
11875,nung1290,Wubuy,gunw1250,Gunwinyguan,Australia,-14.2623,135.753,nuy
17040,bura1267,Burarra,mani1293,Maningrida,Australia,-12.3061,134.365,bvr


In [ ]:
langs=

In [59]:
df_langs.to_csv("../data/stage1/languages.csv", index=False)

In [33]:
import random

In [49]:
possibles= [ "maun1240","nung1290", "bura1267"]

In [48]:
possibles

['yuga1244', 'maun1240', 'nung1290', 'bura1267']

In [56]:
s = get_indryer2018_isos(possibles, "yuga1244", df_langs)

maun1240
297
nung1290
56
bura1267
193


In [51]:
s

set()